In [2]:
import os
import math
import numpy as np
import torch
import torch.nn.functional as F
import Pk_library as PKL
import imageio
import matplotlib
import matplotlib.pyplot as plt

import utilities
import flow_architecture
import losses

In [ ]:
device = 'cuda'
float_dtype = np.float32
torch.set_default_tensor_type(torch.cuda.FloatTensor)
device_id = 0
torch.cuda.set_device(device_id)

In [ ]:
save_dir = "nbody_128px_mask_1p0_flow/"

In [ ]:
class Parameters():
    def __init__(self):
        #Data parameters
        self.nx = 128
        
        #Fitting parameters
        self.nlev_t = 1.0
        self.noise_fac = self.nlev_t
        self.noise_pix = 2*(self.nlev_t)**2
        self.use_ql = False #The nbody power spectrum is matched with trainingdata
        self.wf_batch_size = 100 #The number of maps to fit
        mask128 = (imageio.imread("masks/mask1_128.png")[:, :, 0]/255).astype(float)
        self.mask = mask128 #np.ones((self.nx, self.nx))
        
        #Pre-trained flow parameters
        self.flow_n_layers = 16
        self.flow_hidden = [12, 12]
        
params = Parameters()

In [ ]:
y_true_np = np.load(save_dir + 'true_maps.npy')
y_pred_np = np.load(save_dir + 'masked_maps.npy')
y_pred_flow_cc_np = np.load(save_dir + 'flow_maps.npy')
y_pred_wf_cc_np = np.load(save_dir + 'wf_maps.npy')

In [ ]:
batch_size = y_true_np.shape[0]

In [ ]:
grid    = 128     #the map will have grid^2 pixels
BoxSize = 512.0   #Mpc/h
MAS     = 'None'  #MAS used to create the image; 'NGP', 'CIC', 'TSC', 'PCS' o 'None'
threads = 1       #number of openmp threads

In [ ]:
Pk_flow_mean  = 0
Pk_flow_cc_mean = 0
Pk_wf_mean = 0

with torch.no_grad():
    for i in range(batch_size):
        Pk2D_flow = PKL.Pk_plane(y_pred_flow_cc_np[i], BoxSize, MAS, threads, verbose=False)
        Pk_flow_mean += Pk2D_flow.Pk
        Pk2D_wf = PKL.Pk_plane(y_pred_wf_cc_np[i], BoxSize, MAS, threads, verbose=False)
        Pk_wf_mean += Pk2D_wf.Pk

    Pk_flow_mean /= params.wf_batch_size
    Pk_wf_mean /= params.wf_batch_size
    k = Pk2D_flow.k

plt.plot(k[:-1], Pk_flow_mean[:-1], label='Flow')
plt.plot(k[:-1], Pk_wf_mean[:-1], label='Wiener filtering')
plt.xscale('log')
plt.yscale('log')
plt.legend()

In [ ]:
fft_split_factor = 7.

y_pred_flow_fft = np.fft.fft2(y_pred_flow_cc_np)
y_pred_flow_fft_high = utilities.high_of_fft(y_pred_flow_fft, fft_split_factor)
y_pred_wf_fft = np.fft.fft2(y_pred_wf_cc_np)
y_pred_wf_fft_low = utilities.low_of_fft(y_pred_wf_fft, fft_split_factor)

In [ ]:
y_pred_split_fft = y_pred_flow_fft_high + y_pred_wf_fft_low
y_pred_split = np.fft.ifft2(y_pred_split_fft).real

In [ ]:
#Our simulations originally had the following std and mean
sim_std = 14.3145
sim_mean = 2.23550

In [ ]:
def make_overdensity_from_0mean_1std(array, sim_std, sim_mean):
    """ Input array is 0 mean, 1 std. data.  It is returned to original simulation normalization, then normalized as an overdensity"""
    array = (array + sim_mean)*sim_std
    overdensity = (array - np.mean(array))/np.mean(array)
    return overdensity

In [ ]:
y_true_np = make_overdensity_from_0mean_1std(y_true_np, sim_std, sim_mean)
y_pred_np = make_overdensity_from_0mean_1std(y_pred_np, sim_std, sim_mean)
y_pred_flow_cc_np = make_overdensity_from_0mean_1std(y_pred_flow_cc_np, sim_std, sim_mean)
y_pred_wf_cc_np = make_overdensity_from_0mean_1std(y_pred_wf_cc_np, sim_std, sim_mean)
y_pred_split = make_overdensity_from_0mean_1std(y_pred_split, sim_std, sim_mean)

In [ ]:
vmin = -0.5
vmax = 4.5
figsize = (6, 6)

In [ ]:
utilities.imshow(y_true_np[0], vmin=vmin, vmax=vmax, figsize=figsize, axis=False, colorbar=False, file_name=save_dir+'truth.png')

In [ ]:
utilities.imshow(y_pred_np[0], vmin=vmin, vmax=vmax, figsize=figsize, axis=False, colorbar=False, file_name=save_dir+'masked.png')

In [ ]:
y_pred_np_vmin_mask = y_pred_np[0] - (1-params.mask)*100
utilities.imshow(y_pred_np_vmin_mask, vmin=vmin, vmax=vmax, figsize=figsize, axis=False, colorbar=False, file_name=save_dir+'masked_2.png')

In [ ]:
y_pred_np_vmin_mask = y_pred_np[0] * params.mask

y_pred_np_vmin_mask = np.ma.masked_where(y_pred_np_vmin_mask == 0., y_pred_np_vmin_mask)

plt.figure(figsize=figsize)
cmap = matplotlib.cm.viridis
cmap.set_bad('lightgreen', 1.)
plt.imshow(y_pred_np_vmin_mask, vmin=vmin, vmax=vmax, cmap=cmap)
plt.axis('off')
plt.tight_layout()
plt.savefig(save_dir+'masked_3.png')
plt.show()

In [ ]:
utilities.imshow(y_pred_wf_cc_np[0], vmin=vmin, vmax=vmax, figsize=figsize, axis=False, colorbar=False, file_name=save_dir+'wf.png')

Fourier split

In [ ]:
utilities.imshow(y_pred_split[0], vmin=vmin, vmax=vmax, colorbar=False, figsize=figsize, axis=False, file_name=save_dir+'flow.png')

"""J2_split_mean = 0.
for n in range(params.wf_batch_size):
    J2_split_mean += lossfunctions.loss_J2(y_true[n], torch.tensor(y_pred_split[n])).cpu().detach().numpy()
J2_split_mean = J2_split_mean / params.wf_batch_size
print(J2_split_mean)"""

In [ ]:
print(np.mean((y_pred_split.astype('float32') - y_true_np.astype('float32'))**2))
print(np.mean((y_pred_wf_cc_np.astype('float32') - y_true_np.astype('float32'))**2))

In [ ]:
print(np.mean(((y_pred_split.astype('float32') - y_true_np.astype('float32'))*params.mask)**2))
print(np.mean(((y_pred_wf_cc_np.astype('float32') - y_true_np.astype('float32'))*params.mask)**2))

In [ ]:
interior_frac = np.sum(params.mask) / (params.nx**2)

In [ ]:
print(np.mean(((y_pred_split.astype('float32') - y_true_np.astype('float32'))*params.mask)**2) / interior_frac)
print(np.mean(((y_pred_wf_cc_np.astype('float32') - y_true_np.astype('float32'))*params.mask)**2) / interior_frac)

In [ ]:
plt.rcParams.update({'font.size': 14})

In [ ]:
Pk_true = np.zeros((batch_size, 90))
Pk_flow = np.zeros((batch_size, 90))
Pk_flow_cc = np.zeros((batch_size, 90))
Pk_wf = np.zeros((batch_size, 90))
Pk_wf_cc = np.zeros((batch_size, 90))
Pk_noise = np.zeros((batch_size, 90))
N_flow = np.zeros((batch_size, 90))
N_wf = np.zeros((batch_size, 90))

with torch.no_grad():
    for i in range(batch_size):
        Pk2D_flow_true = PKL.XPk_plane((y_pred_split[i]*params.mask).astype('float32'), (y_true_np[i]*params.mask).astype('float32'), BoxSize, MAS, MAS, threads)
        Pk_flow[i] = Pk2D_flow_true.Pk[:, 0]
        Pk_true[i] = Pk2D_flow_true.Pk[:, 1]
        Pk_flow_cc[i] = Pk2D_flow_true.r
        
        Pk2D_wf_true = PKL.XPk_plane((y_pred_wf_cc_np[i]*params.mask).astype('float32'), (y_true_np[i]*params.mask).astype('float32'), BoxSize, MAS, MAS, threads)
        Pk_wf[i] = Pk2D_wf_true.Pk[:, 0]
        Pk_wf_cc[i] = Pk2D_wf_true.r
        
        noise = y_pred_np[i] - y_true_np[i]
        Pk2D_noise = PKL.Pk_plane((noise*params.mask).astype('float32'), BoxSize, MAS, threads)
        Pk_noise[i] = Pk2D_noise.Pk
        
        epsilon_flow = y_pred_split[i] - y_true_np[i]
        Pk2D_Nflow = PKL.Pk_plane((epsilon_flow*params.mask).astype('float32'), BoxSize, MAS, threads)
        N_flow[i] = Pk2D_Nflow.Pk
        
        epsilon_wf = y_pred_wf_cc_np[i] - y_true_np[i]
        Pk2D_Nwf = PKL.Pk_plane((epsilon_wf*params.mask).astype('float32'), BoxSize, MAS, threads)
        N_wf[i] = Pk2D_Nwf.Pk
        
    kvals = Pk2D_flow_true.k


rcut = 4

kvals = kvals[:-rcut]
    
Pk_flow_cc_mean = np.mean(Pk_flow_cc[:, :-rcut], 0)
Pk_flow_cc_1sigma = np.std(Pk_flow_cc[:, :-rcut], 0)
Pk_wf_cc_mean = np.mean(Pk_wf_cc[:, :-rcut], 0)
Pk_wf_cc_1sigma = np.std(Pk_wf_cc[:, :-rcut], 0)

Pk_figsize = (5, 4)
plt.figure(figsize=Pk_figsize)
plt.plot(kvals, Pk_flow_cc_mean, label=r'$r^{\mathrm{true, flow}}$')
plt.fill_between(kvals, Pk_flow_cc_mean+Pk_flow_cc_1sigma, Pk_flow_cc_mean-Pk_flow_cc_1sigma, alpha=0.2)
plt.plot(kvals, Pk_wf_cc_mean, label=r'$r^{\mathrm{true, Wiener\ filtered}}$')
plt.fill_between(kvals, Pk_wf_cc_mean+Pk_wf_cc_1sigma, Pk_wf_cc_mean-Pk_wf_cc_1sigma, alpha=0.2)
plt.xscale('log')
plt.xlabel(r'$k\ (h/\mathrm{Mpc})$')
plt.ylabel(r'$r(k)$')
plt.legend(loc='lower left')
#plt.tight_layout()
plt.savefig(save_dir + 'r_0p1noise_mask_small.pdf', bbox_inches='tight')
plt.show()


Pk_true_mean = np.mean(Pk_true[:, :-rcut], 0)
Pk_true_1sigma = np.std(Pk_true[:, :-rcut], 0)
Pk_flow_mean = np.mean(Pk_flow[:, :-rcut], 0)
Pk_flow_1sigma = np.std(Pk_flow[:, :-rcut], 0)
Pk_wf_mean = np.mean(Pk_wf[:, :-rcut], 0)
Pk_wf_1sigma = np.std(Pk_wf[:, :-rcut], 0)
Pk_noise_mean = np.mean(Pk_noise[:, :-rcut], 0)
Pk_noise_1sigma = np.std(Pk_noise[:, :-rcut], 0)
N_flow_mean = np.mean(N_flow[:, :-rcut], 0)
N_flow_1sigma = np.std(N_flow[:, :-rcut], 0)
N_wf_mean = np.mean(N_wf[:, :-rcut], 0)
N_wf_1sigma = np.std(N_wf[:, :-rcut], 0)

plt.figure(figsize=Pk_figsize)
plt.plot(kvals, Pk_flow_mean, label='Flow posterior')
plt.fill_between(kvals, Pk_flow_mean+Pk_flow_1sigma, Pk_flow_mean-Pk_flow_1sigma, alpha=0.2)
plt.plot(kvals, Pk_wf_mean, label='Wiener filtered')
plt.fill_between(kvals, Pk_wf_mean+Pk_wf_1sigma, Pk_wf_mean-Pk_wf_1sigma, alpha=0.2)
plt.plot(kvals, Pk_true_mean, label='Truth', linestyle='--')
plt.fill_between(kvals, Pk_true_mean+Pk_true_1sigma, Pk_true_mean-Pk_true_1sigma, alpha=0.2)
plt.plot(kvals, Pk_noise_mean, label=r'$1.0\tilde{\sigma}\ \mathrm{noise}$')
plt.fill_between(kvals, Pk_noise_mean+Pk_noise_1sigma, Pk_noise_mean-Pk_noise_1sigma, alpha=0.2)
plt.plot(kvals, N_flow_mean, label=r'$N^{\mathrm{flow}}$')
plt.fill_between(kvals, N_flow_mean+N_flow_1sigma, N_flow_mean-N_flow_1sigma, alpha=0.2)
plt.plot(kvals, N_wf_mean, label=r'$N^{\mathrm{Wiener\ filtered}}$')
plt.fill_between(kvals, N_wf_mean+N_wf_1sigma, N_wf_mean-N_wf_1sigma, alpha=0.2)
plt.xscale('log')
plt.yscale('log')
plt.xlabel(r'$k\ (h/\mathrm{Mpc})$')
plt.ylabel(r'$P(k)\ (\mathrm{Mpc}^2/h^2)$')
plt.legend(loc='lower left')
#plt.tight_layout()
plt.savefig(save_dir + 'ps_0p1noise_mask_small.pdf', bbox_inches='tight')
plt.show()